# **Business Understanding**

### I. Problem Statements
1. Bagaimana sistem dapat merekomendasikan buku-buku yang paling populer dan disukai secara umum oleh pengguna?
2. Dapatkah sistem menyaring dan merekomendasikan buku berkualitas tinggi berdasarkan penilaian dari reviewer ahli (pengguna aktif)?

### II. Goals
1. Menghasilkan daftar rekomendasi buku berdasarkan popularitas.
2. Menghasilkan daftar rekomendasi buku yang terkurasi, berdasarkan rating dari reviewer ahli yang telah memberikan ulasan dalam jumlah signifikan.

### III. Solution Statements
- Mengembangkan sistem rekomendasi berbasis popularitas (Content Based).
- Mengembangkan sistem filtering berbasis reviewer ahli dengan cara menyaring ulasan dari reviewer ahli (Filtering Based).
- Menyusun dua daftar rekomendasi:
1. Buku terpopuler secara umum.
2. Buku dengan rating tertinggi dari reviewer ahli.
- Sistem ini tidak bergantung pada histori personal pengguna sehingga sangat cocok untuk mengatasi masalah cold-start (pengguna baru).

# **Load Dataset**

In [ ]:
!pip install numpy==1.26.4 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 265.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import Image, display
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
print(np.__version__)

1.26.4


In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2469536 sha256=06e7695a699ad1b4deafbc5d168894b2c379a31eb614f71aab8653f635ebd0a9
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fitriairfani","key":"1c942ff3a36c8f362a9e2979f6e63f0f"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Unduh dan ekstrak dataset langsung
!kaggle datasets download -d arashnic/book-recommendation-dataset
!unzip -q book-recommendation-dataset.zip -d dataset

Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
  0% 0.00/24.3M [00:00<?, ?B/s]
100% 24.3M/24.3M [00:00<00:00, 672MB/s]


In [ ]:
# Load data from CSV files
books = pd.read_csv('dataset/Books.csv')
users = pd.read_csv('dataset/Users.csv')
ratings = pd.read_csv('dataset/Ratings.csv')

<ipython-input-7-2472819329>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('dataset/Books.csv')


# **Data Understanding**

## EDA
## Cek Informasi Tiap Variabel

Dataset yang digunakan terdiri dari 3 file berikut:
1. **Users:** Berisi fitur ID pengguna (`User-ID`), Lokasi Pengguna (`Location`), dan Usia (`Age`).
2.  **Books:** Berisi fitur `ISBN`, Judul Buku (`Book-Title`), Penulis Buku (`Book-Author`), Tahun Terbit (`Year-Of-Publication`), Penerbit (`Publisher`), dan URL yang menautkan ke gambar sampul terdapat tiga bentuk berbeda (`Image-URL-S`, `Image-URL-M`, dan `Image-URL-L`).
3. **Ratings:** Berisi fitur ID pengguna (`User-ID`), `ISBN`, dan Hasil Penilaian (`Book-Rating`) yang dinyatakan dalam skala 1-10 (nilai yang lebih tinggi menunjukkan apresiasi yang lebih tinggi).

### Users Variable

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [ ]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Dataset `users` berisi informasi tentang pengguna, terdiri dari **278.858 baris** dan **3 kolom** dengan fitur:
1. **User-ID** bertipe data Integer. Merupakan identifikasi unik untuk setiap pengguna.
2. **Location** bertipe data Object (string). Berisi informasi lokasi pengguna dalam format `kota, provinsi, negara`.
3. **Age** bertipe data Float. Usia pengguna dicatat dalam satuan tahun.

### Books Variabel

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [ ]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Dataset `books` terdiri dari **271.360 entri** dengan **8 kolom** yang semuanya bertipe *object* (string). Kolom-kolom ini mencakup informasi seperti:
1. `ISBN`: Nomor unik identifikasi buku.
2. `Book-Title`: Judul buku.
3. `Book-Author`: Nama penulis buku (2 nilai kosong).
4. `Year-Of-Publication`: Tahun terbit buku.
5. `Publisher`: Nama penerbit (2 nilai kosong).
6. `Image-URL-S`, `Image-URL-M`, `Image-URL-L`: URL gambar buku dalam tiga ukuran berbeda (3 nilai kosong pada URL-L).

Secara keseluruhan, dataset ini cukup bersih dan kaya akan informasi metadata buku yang bermanfaat untuk sistem rekomendasi berbasis konten maupun kolaboratif.

### Ratings Variable

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Dataset `ratings` terdiri dari **1.149.780 entri** dengan **3 kolom**:
1. `User-ID`: Identifikasi pengguna (int64).
2. `ISBN`: Identifikasi buku (object).
3. `Book-Rating`: Nilai rating yang diberikan (int64).

In [ ]:
ratings['Book-Rating'].describe()

,Book-Rating
count,1.149780e+06
mean,2.866950e+00
std,3.854184e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,7.000000e+00
max,1.000000e+01


Semua kolom tidak memiliki nilai kosong. Statistik dari kolom `Book-Rating` menunjukkan:
- **Mean**: 2.87
- **Standard Deviation**: 3.85
- **Minimum**: 0
- **Maximum**: 10
- **50% nilai rating (median)**: 0
    
Mayoritas nilai rating bernilai 0, mengindikasikan bahwa sebagian besar entri kemungkinan merupakan *implicit feedback* (misalnya buku dilihat tetapi tidak diberi rating eksplisit). Hal ini menunjukkan bahwa data rating bersifat sparse dan perlu penanganan lanjutan, seperti pemisahan antara rating eksplisit dan implisit, untuk meningkatkan kualitas hasil rekomendasi.

## Cek Missing Value

Dilakukan pengecekan missing value pada data `books`, karena data `books` merupakan data utama yang digunakan.

In [ ]:
books.isna().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


Dari hasil pengecekan mising value di atas, terdapat sebanyak 7 data, dengan perincian 2 data kosong pada fitur `Book-Author`, 2 data kosong pada fitur dan perlu dilakukan suatu penanganan.

In [ ]:
books[books.isnull().any(axis=1)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


## Cek Duplikasi Data

Dilakukan pengecekan apakah terdapat duplikasi data pada data `books`sebagai data utama.

In [ ]:
#  Mengecek duplikasi pada dataframe books
books.duplicated().sum()

0

Dari hasil pengecekan duplikasi data pada data `books`, tidak ditemukan adanya data yang duplikasi sehingga tidak diperlukan suatu penanganan.

## Hasil EDA

Dari hasil EDA yang dilakukan, dapat disimpulkan bahwa:
1. **Dataset yang digunakan**:
  - `ratings`: memberikan informasi tentang penilaian pengguna terhadap buku.
  - `books`: memberikan metadata buku, termasuk judul buku (`Book-Title`) yang digunakan sebagai kunci analisis.
2. **Dataset yang tidak digunakan**:
  - `users`: tidak dilibatkan dalam proses ini karena fokus analisis adalah pada buku, bukan pengguna.

# **Data Preparation**

## Penanganan Missing Value

Pada EDA sebelumnya, diketahui bahwa data `books` terdapat missing value dan diperlukan penanganan agar mempermudah proses analisis.

 Digunakan penanganan **dropping missing values** dikarenakan jumlah data yang hilang pada data `books` sedikit dan apabila dilakukan penghapusan data maka hasil analisis tidak akan terpengaruh secara signifikan.

In [ ]:
books.dropna(inplace = True)

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271353 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271353 non-null  object
 1   Book-Title           271353 non-null  object
 2   Book-Author          271353 non-null  object
 3   Year-Of-Publication  271353 non-null  object
 4   Publisher            271353 non-null  object
 5   Image-URL-S          271353 non-null  object
 6   Image-URL-M          271353 non-null  object
 7   Image-URL-L          271353 non-null  object
dtypes: object(8)
memory usage: 18.6+ MB


Setelah dilakukan dropping missing values data `books` yang awalnya memiliki jumlah data sebanyak 271.360 menjadi 271.353 data.

## Menggabungkan Data Buku dengan Data Penilaian berdasarkan 'ISBN'

Langkah ini menggabungkan dataframe `ratings` dan `books` berdasarkan kolom ISBN. Hasilnya adalah dataframe `ratings_with_name` yang berisi data rating yang telah dilengkapi informasi buku seperti judul, penulis, dan penerbit.

In [ ]:
# Menggabungkan dataframe ratings dan books berdasarkan nilai 'ISBN'
ratings_with_name = ratings.merge(books, on='ISBN')
ratings_with_name

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...
...,...,...,...,...,...,...,...,...,...,...
1031123,276704,0876044011,0,Edgar Cayce on the Akashic Records: The Book o...,Kevin J. Todeschi,1998,A.R.E. Press (Association of Research &amp; Enlig,http://images.amazon.com/images/P/0876044011.0...,http://images.amazon.com/images/P/0876044011.0...,http://images.amazon.com/images/P/0876044011.0...
1031124,276704,1563526298,9,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard,2000,Longstreet Press,http://images.amazon.com/images/P/1563526298.0...,http://images.amazon.com/images/P/1563526298.0...,http://images.amazon.com/images/P/1563526298.0...
1031125,276706,0679447156,0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil,1997,Alfred A. Knopf,http://images.amazon.com/images/P/0679447156.0...,http://images.amazon.com/images/P/0679447156.0...,http://images.amazon.com/images/P/0679447156.0...
1031126,276709,0515107662,10,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter,1996,Jove Books,http://images.amazon.com/images/P/0515107662.0...,http://images.amazon.com/images/P/0515107662.0...,http://images.amazon.com/images/P/0515107662.0...


In [ ]:
ratings_with_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031128 entries, 0 to 1031127
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   User-ID              1031128 non-null  int64 
 1   ISBN                 1031128 non-null  object
 2   Book-Rating          1031128 non-null  int64 
 3   Book-Title           1031128 non-null  object
 4   Book-Author          1031128 non-null  object
 5   Year-Of-Publication  1031128 non-null  object
 6   Publisher            1031128 non-null  object
 7   Image-URL-S          1031128 non-null  object
 8   Image-URL-M          1031128 non-null  object
 9   Image-URL-L          1031128 non-null  object
dtypes: int64(2), object(8)
memory usage: 78.7+ MB


## Train Test Data untuk Content Filtering

Tahapan ini membagi data interaksi pengguna dan buku (`ratings_with_name`) menjadi dua bagian: `train_data` (80%) untuk melatih sistem dan `test_data` (20%) untuk menguji efektivitas rekomendasi.

In [ ]:
from sklearn.model_selection import train_test_split

# Pisahkan data interaksi menjadi train & test
train_data, test_data = train_test_split(ratings_with_name, test_size=0.2)

In [ ]:
# Proses untuk train_data
num_rating_train = train_data.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_train.rename(columns={'Book-Rating':'Num-Ratings'}, inplace=True)

avg_rating_train = (
    train_data.groupby('Book-Title')['Book-Rating']
    .mean()
    .reset_index()
)
avg_rating_train.rename(columns={'Book-Rating': 'Avg-Rating'}, inplace=True)

popular_books_train = num_rating_train.merge(avg_rating_train, on="Book-Title")

most_popular_books_train = popular_books_train[popular_books_train['Num-Ratings'] >= 400].sort_values('Avg-Rating', ascending=False)

most_popular_books_train = most_popular_books_train.merge(books, on="Book-Title").drop_duplicates('Book-Title')[
    ['Book-Title', 'Book-Author', 'Image-URL-M', 'Num-Ratings', 'Avg-Rating']
]

In [ ]:
# Proses untuk test_data
num_rating_test = test_data.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_test.rename(columns={'Book-Rating':'Num-Ratings'}, inplace=True)

avg_rating_test = (
    test_data.groupby('Book-Title')['Book-Rating']
    .mean()
    .reset_index()
)
avg_rating_test.rename(columns={'Book-Rating': 'Avg-Rating'}, inplace=True)

popular_books_test = num_rating_test.merge(avg_rating_test, on="Book-Title")

most_popular_books_test = popular_books_test[popular_books_test['Num-Ratings'] >= 80].sort_values('Avg-Rating', ascending=False)

most_popular_books_test = most_popular_books_test.merge(books, on="Book-Title").drop_duplicates('Book-Title')[
    ['Book-Title', 'Book-Author', 'Image-URL-M', 'Num-Ratings', 'Avg-Rating']
]

In [ ]:
# Inisialisasi hasil train dan test
relevant = most_popular_books_train['Book-Title'].head(10).tolist()
recommended = most_popular_books_test['Book-Title'].head(10).tolist()

**Hasil Akhir:**

- `relevant` merepresentasikan daftar buku populer dari data training, yang dianggap relevan sebagai acuan sistem rekomendasi.
- `recommended` merepresentasikan daftar buku rekomendasi hasil dari model menggunakan data testing.

Tujuannya adalah untuk membandingkan hasil rekomendasi model (recommended) dengan buku-buku yang memang populer dan disukai pengguna (relevant) dari data pelatihan, sebagai bagian dari proses evaluasi sistem.

## Mengambil Data Buku Populer Berdasarkan Reviewer Ahli dan Rating

Tujuan dari proses ini adalah untuk meningkatkan kualitas data yang digunakan dalam sistem rekomendasi dengan cara memfokuskan analisis hanya pada pengguna yang aktif (experienced users) dan buku yang populer di kalangan mereka. Dengan menyaring pengguna yang telah memberikan lebih dari 200 penilaian, sistem hanya mempertimbangkan opini dari reviewer yang memiliki pengalaman membaca yang cukup luas, sehingga penilaiannya dianggap lebih kredibel.

Selanjutnya, hanya buku yang telah menerima setidaknya 100 rating dari para pengguna aktif ini yang dipertahankan. Langkah ini menghindari buku-buku yang jarang dinilai (yang bisa menyebabkan bias atau noise) dan memastikan bahwa rekomendasi didasarkan pada data yang kuat dan representatif, baik dari sisi pengguna maupun buku.

In [ ]:
# Mengambil data User-ID dari data users yang telah menilai lebih dari 200 buku
x = ratings_with_name.groupby("User-ID").count()['Book-Rating'] > 200
experienced_users = x[x].index

In [ ]:
# Melihat profil buku yang dinilai oleh experienved_users
filtered_ratings = ratings_with_name[ratings_with_name['User-ID'].isin(experienced_users)]
filtered_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1150,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1151,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,http://images.amazon.com/images/P/0026217457.0...,http://images.amazon.com/images/P/0026217457.0...
1152,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,http://images.amazon.com/images/P/003008685X.0...,http://images.amazon.com/images/P/003008685X.0...
1153,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,http://images.amazon.com/images/P/0030615321.0...,http://images.amazon.com/images/P/0030615321.0...
1154,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,http://images.amazon.com/images/P/0060002050.0...,http://images.amazon.com/images/P/0060002050.0...
...,...,...,...,...,...,...,...,...,...,...
1029349,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...,http://images.amazon.com/images/P/1931868123.0...
1029350,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...,http://images.amazon.com/images/P/3411086211.0...,http://images.amazon.com/images/P/3411086211.0...
1029351,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...,http://images.amazon.com/images/P/3829021860.0...,http://images.amazon.com/images/P/3829021860.0...
1029352,275970,4770019572,0,Musashi,Eiji Yoshikawa,1995,Kodansha International (JPN),http://images.amazon.com/images/P/4770019572.0...,http://images.amazon.com/images/P/4770019572.0...,http://images.amazon.com/images/P/4770019572.0...


In [ ]:
# Buku yang memiliki setidaknya 100 penilaian
y = filtered_ratings.groupby('Book-Title').count()['Book-Rating']>=100
famous_books = y[y].index

In [ ]:
final_ratings = filtered_ratings[filtered_ratings['Book-Title'].isin(famous_books)]

In [ ]:
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1163,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
1165,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...
1168,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...,http://images.amazon.com/images/P/0061009059.0...
1188,277427,0142001740,0,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books,http://images.amazon.com/images/P/0142001740.0...,http://images.amazon.com/images/P/0142001740.0...,http://images.amazon.com/images/P/0142001740.0...
1206,277427,0312966091,0,Three To Get Deadly : A Stephanie Plum Novel (...,Janet Evanovich,1998,St. Martin's Paperbacks,http://images.amazon.com/images/P/0312966091.0...,http://images.amazon.com/images/P/0312966091.0...,http://images.amazon.com/images/P/0312966091.0...
...,...,...,...,...,...,...,...,...,...,...
1028786,275970,0618002227,0,The Fellowship of the Ring (The Lord of the Ri...,J. R. R. Tolkien,1999,Houghton Mifflin Company,http://images.amazon.com/images/P/0618002227.0...,http://images.amazon.com/images/P/0618002227.0...,http://images.amazon.com/images/P/0618002227.0...
1028805,275970,0670032379,0,The Secret Life of Bees,Sue Monk Kidd,2002,Viking Books,http://images.amazon.com/images/P/0670032379.0...,http://images.amazon.com/images/P/0670032379.0...,http://images.amazon.com/images/P/0670032379.0...
1028857,275970,0679442790,0,The Reader,Bernhard Schlink,1997,Random House,http://images.amazon.com/images/P/0679442790.0...,http://images.amazon.com/images/P/0679442790.0...,http://images.amazon.com/images/P/0679442790.0...
1028891,275970,0679893105,0,"The Golden Compass (His Dark Materials, Book 1)",PHILIP PULLMAN,1998,Knopf Books for Young Readers,http://images.amazon.com/images/P/0679893105.0...,http://images.amazon.com/images/P/0679893105.0...,http://images.amazon.com/images/P/0679893105.0...


**Hasil Akhir:**

Dataframe `final_ratings` berisi buku-buku yang dianggap populer, yaitu buku yang telah menerima setidaknya 100 rating dari pengguna berpengalaman (pengguna yang telah memberi lebih dari 200 rating).

# **Modeling dan Hasil**

Pada tahap modeling ini akan digunakan dua pendekatan sistem rekomendasi yang saling melengkapi untuk menghasilkan hasil yang lebih robust, yaitu:

1. **Content-Based Filtering**
  
  Pendekatan ini digunakan untuk merekomendasikan buku berdasarkan karakteristik atau atribut tertentu dari buku itu sendiri, dalam hal ini jumlah rating terbanyak dan rata-rata rating tertinggi. Buku yang sering dinilai dan mendapatkan rating tinggi diasumsikan memiliki kualitas atau daya tarik yang tinggi di mata banyak pengguna, sehingga layak untuk direkomendasikan secara umum. Pendekatan ini sangat berguna terutama saat informasi interaksi pengguna masih terbatas atau ketika ingin memberikan rekomendasi populer yang bersifat umum.

2. **Collaborative-Based Filtering (menggunakan SVD)**
    
  Pendekatan ini memanfaatkan data historis interaksi pengguna–dalam bentuk rating–untuk mempelajari pola preferensi dan merekomendasikan buku yang serupa dengan buku yang disukai oleh pengguna lain yang memiliki pola perilaku serupa. Dengan menerapkan algoritma Singular Value Decomposition (SVD), sistem dapat mengisi kekosongan data (sparse matrix) dan memprediksi rating yang mungkin diberikan oleh pengguna terhadap buku yang belum mereka nilai. Pendekatan ini lebih bersifat personal dan akurat karena mempertimbangkan pengalaman pengguna lain sebagai dasar rekomendasi.

Dengan menggabungkan kedua pendekatan ini, sistem dapat memberikan rekomendasi buku yang tidak hanya populer secara umum tetapi juga relevan secara personal bagi masing-masing pengguna.

## Content Filtering Based Recommendation System

Pada proses ini, digunakan pendekatan sederhana berbasis **content filtering** yang menyeleksi buku-buku berdasarkan dua fitur utama: **jumlah rating terbanyak** dan **rata-rata rating tertinggi**. Pendekatan ini mengasumsikan bahwa buku yang sering dinilai dan mendapat rating tinggi cenderung disukai oleh banyak pengguna, sehingga layak untuk direkomendasikan.

In [ ]:
# Melihat 10 profil buku rekomendasi yang memiliki penilaian terbanyak dengan nilai tertinggi
for i in range(10):
    print(f"\nBuku #{i+1}")
    print("Judul        :", most_popular_books_test['Book-Title'].iloc[i])
    print("Penulis      :", most_popular_books_test['Book-Author'].iloc[i])
    print("Jumlah Rating:", most_popular_books_test['Num-Ratings'].iloc[i])
    print("Rata-rata    :", round(most_popular_books_test['Avg-Rating'].iloc[i], 2))
    display(Image(url=most_popular_books_test['Image-URL-M'].iloc[i]))


Buku #1
Judul        : Harry Potter and the Prisoner of Azkaban (Book 3)
Penulis      : J. K. Rowling
Jumlah Rating: 88
Rata-rata    : 6.12



Buku #2
Judul        : Harry Potter and the Chamber of Secrets (Book 2)
Penulis      : J. K. Rowling
Jumlah Rating: 125
Rata-rata    : 5.42



Buku #3
Judul        : To Kill a Mockingbird
Penulis      : Harper Lee
Jumlah Rating: 103
Rata-rata    : 5.26



Buku #4
Judul        : The Catcher in the Rye
Penulis      : J.D. Salinger
Jumlah Rating: 105
Rata-rata    : 5.03



Buku #5
Judul        : Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson
Penulis      : MITCH ALBOM
Jumlah Rating: 100
Rata-rata    : 4.94



Buku #6
Judul        : Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Penulis      : J. K. Rowling
Jumlah Rating: 111
Rata-rata    : 4.93



Buku #7
Judul        : The Five People You Meet in Heaven
Penulis      : Mitch Albom
Jumlah Rating: 82
Rata-rata    : 4.85



Buku #8
Judul        : The Da Vinci Code
Penulis      : Dan Brown
Jumlah Rating: 178
Rata-rata    : 4.76



Buku #9
Judul        : Life of Pi
Penulis      : Yann Martel
Jumlah Rating: 142
Rata-rata    : 4.7



Buku #10
Judul        : Fahrenheit 451
Penulis      : Ray Bradbury
Jumlah Rating: 81
Rata-rata    : 4.62


## Collaborative Filtering Based Recommendation System (SVD Modeling)

Collaborative Filtering merupakan salah satu pendekatan paling populer dalam sistem rekomendasi yang berfokus pada pola interaksi antar pengguna dan item (dalam hal ini, buku). Pendekatan ini tidak memerlukan informasi konten dari buku, melainkan hanya mengandalkan rating atau feedback yang diberikan pengguna. Prinsip utamanya adalah: jika dua pengguna memiliki preferensi yang mirip di masa lalu, maka mereka cenderung menyukai item yang sama di masa depan.

Dalam proyek ini, collaborative filtering diterapkan menggunakan algoritma Singular Value Decomposition (SVD) dari library Surprise. SVD mampu mengatasi permasalahan umum dalam data rating seperti sparsity (banyaknya nilai kosong) dengan memetakan pengguna dan item ke dalam dimensi laten. Hasilnya adalah model yang dapat memprediksi seberapa besar kemungkinan seorang pengguna akan menyukai buku tertentu, meskipun belum pernah memberi rating untuk buku tersebut.

In [ ]:
from surprise.model_selection import train_test_split

# Define the rating scale
reader = Reader(rating_scale=(0, 10))

# Load the data into Surprise's dataset format
data = Dataset.load_from_df(final_ratings[['User-ID', 'Book-Title', 'Book-Rating']], reader)

# Split the dataset into training and testing sets
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

# Define the SVD algorithm
model = SVD()

# Train the algorithm on the training set
model.fit(train_set)

In [ ]:
def recommend_books(user_id, n=10):
    # List all unique book titles
    all_books = final_ratings['Book-Title'].unique()

    # Remove books already rated by the user
    rated_books = final_ratings[final_ratings['User-ID'] == user_id]['Book-Title'].values
    books_to_predict = [book for book in all_books if book not in rated_books]

    # Predict ratings for remaining books
    predictions = []
    for book in books_to_predict:
        pred = model.predict(user_id, book)
        predictions.append((book, pred.est))

    # Sort predictions by estimated rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_n = predictions[:n]

    return top_n

In [ ]:
from IPython.display import Image, display

# Make book recommendations
user_id = 271705
recommended_books = recommend_books(user_id)

print(f"Top 10 recommended books for user {user_id}:\n")
for i, (title, _) in enumerate(recommended_books, start=1):
    # Filter baris data berdasarkan judul
    book_info = final_ratings[final_ratings['Book-Title'] == title].iloc[0]

    # Ambil nama penulis dan URL gambar
    author_name = book_info['Book-Author'] if 'Book-Author' in book_info else "Tidak diketahui"
    image_url = book_info['Image-URL-M'] if 'Image-URL-M' in book_info else None

    # Tampilkan informasi
    print(f"Buku #{i}")
    print(f"Judul  : {title}")
    print(f"Penulis: {author_name}\n")

    # Tampilkan gambar jika URL tersedia
    if image_url:
        display(Image(url=image_url))

Top 10 recommended books for user 271705:

Buku #1
Judul  : Harry Potter and the Chamber of Secrets (Book 2)
Penulis: J. K. Rowling



Buku #2
Judul  : Harry Potter and the Goblet of Fire (Book 4)
Penulis: J. K. Rowling



Buku #3
Judul  : Harry Potter and the Order of the Phoenix (Book 5)
Penulis: J. K. Rowling



Buku #4
Judul  : The Firm
Penulis: John Grisham



Buku #5
Judul  : The Prince of Tides
Penulis: Pat Conroy



Buku #6
Judul  : The Gift
Penulis: Madeena S. Nolan



Buku #7
Judul  : The Color Purple
Penulis: Alice Walker



Buku #8
Judul  : Interview with the Vampire
Penulis: Anne Rice



Buku #9
Judul  : All I Really Need to Know
Penulis: ROBERT FULGHUM



Buku #10
Judul  : A Painted House
Penulis: JOHN GRISHAM



# **Evaluation**

Evaluasi dilakukan untuk mengukur kinerja dan efektivitas dari dua pendekatan sistem rekomendasi yang telah dibangun, yaitu Content-Based Filtering dan Collaborative Filtering berbasis SVD. Masing-masing pendekatan memiliki karakteristik dan metrik evaluasi yang berbeda, disesuaikan dengan tujuan dan data yang digunakan.

## 1. **Evaluasi Content-Based Filtering**

  Pada pendekatan content-based, sistem merekomendasikan buku berdasarkan jumlah rating terbanyak dan rata-rata rating tertinggi. Evaluasi dilakukan menggunakan dua metrik umum yaitu:
  
  1. Precision@K
  
  Precision@K mengukur seberapa relevan hasil rekomendasi yang diberikan, dengan rumus:

  $$
  \text{Precision@K} = \frac{|\text{Recommended}_k \cap \text{Relevant}|}{K}
  $$

  Semakin tinggi nilai precision, maka semakin besar proporsi buku yang direkomendasikan benar-benar relevan.

  2. Recall@K

  Recall@K mengukur seberapa banyak dari total item relevan yang berhasil direkomendasikan oleh sistem:

  $$
  \text{Recall@K} = \frac{|\text{Recommended}_k \cap \text{Relevant}|}{|\text{Relevant}|}
  $$

  Nilai recall yang tinggi menunjukkan sistem mampu menjangkau sebagian besar buku yang relevan.

## 2. **Evaluasi Collaborative-Based Filtering (SVD)**

  Berbeda dari pendekatan sebelumnya, collaborative filtering berbasis Singular Value Decomposition (SVD) tidak bergantung pada informasi konten buku, melainkan memanfaatkan interaksi historis antar pengguna dan item. Evaluasi dilakukan menggunakan metrik Root Mean Squared Error (RMSE), yang menghitung seberapa jauh prediksi sistem dari nilai rating aktual.

  Rumus matematis RMSE adalah:
  $$
  \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)^2}
  $$

  Di mana:  
  - $n$ adalah jumlah total prediksi,  
  - $\hat{y}_i$ adalah nilai prediksi ke-i,  
  - $y_i$ adalah nilai aktual ke-i.

## Evaluation untuk Content Filtering

In [ ]:
def precision_at_k(recommended_items, relevant_items, k):
    top_k = recommended_items[:k]
    relevant = set(top_k) & set(relevant_items)
    return len(relevant) / k

def recall_at_k(recommended_items, relevant_items, k):
    top_k = recommended_items[:k]
    relevant = set(top_k) & set(relevant_items)
    return len(relevant) / len(relevant_items)

In [ ]:
print("Precision@3:", precision_at_k(recommended, relevant, 10))
print("Recall@3:", recall_at_k(recommended, relevant, 10))

Precision@3: 0.5
Recall@3: 0.5


**Hasil Akhir:**

- Precision@10 = 0.5 → 5 dari 10 rekomendasi termasuk buku relevan.
- Recall@10 = 0.5 → Dari 10 buku relevan, 5 berhasil direkomendasikan sistem.
- Metrik ini menunjukkan bahwa sistem mampu memberikan rekomendasi yang cukup tepat dan relevan terhadap preferensi pengguna berdasarkan karakteristik buku (misalnya judul dan genre).

## Evaluation untuk Collaborative Filtering dengan SVD

In [ ]:
# Make predictions on the test set
predictions = model.test(test_set)

# Evaluate the model
accuracy.rmse(predictions)

RMSE: 3.5880


3.5879665729920953

**Hasil Akhir**

- Nilai **RMSE yang lebih kecil** menunjukkan bahwa prediksi model lebih dekat dengan data aktual.
- Dalam hasil di atas, nilai **RMSE = 3.5880**, artinya secara rata-rata, prediksi model menyimpang sekitar 3.59 satuan dari nilai sebenarnya.
- Meskipun sistem dapat mempelajari pola dari data interaksi pengguna, nilai RMSE tersebut tergolong cukup besar, yang berarti akurasi prediksi rating masih kurang optimal. Hal ini bisa disebabkan oleh data yang sparse (jarang) atau kurangnya informasi pengguna/buku yang cukup beragam.

# Penutup

### Kesimpulan

Content-Based Filtering memberikan hasil yang lebih terkontrol dan dapat dijelaskan (explainable), karena berdasarkan informasi buku yang sudah diketahui.

Collaborative Filtering (SVD) berpotensi memberikan rekomendasi yang lebih personal, tetapi lebih rentan terhadap masalah data sparsity dan cold-start (pengguna/buku baru).

### Saran

Untuk hasil yang lebih optimal, dapat dipertimbangkan pendekatan hybrid yang menggabungkan kedua metode agar saling melengkapi kekurangan masing-masing.